# Capstone Project - IBM Data Science
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

As Munich developed and grew, more and more suburbs and villages of the surrounding area have become an integral part of the Bavarian capital and the third largest city in Germany. Today, the city extends to an area of some 31.1 hectares. The largest expansion of the urban area is 20.7 kilometers from north to south and from east to west 26.9 km. The extent of the city limits is 119 km. Munich is divided into 25 administrative districts. The data in [this page](http://www.total-munich.com/20160623888/blog/moving-to-munich/moving-to-munich-introduction-to-munich-s-boroughs.html) shows all of the city’s 25 districts, as well as their respective surface and population. The data also shows the percentage of foreigners living in each of the districts and the density of the population in each of them.

#### Business Problen

As a Capstone Project for the IBM Data Science professional course, I decided to explore the districts of **Munich**, Germany. And try to find which district would be optimal to open an **Indian restaurant**.

Since there are lots of restaurants in Munich I will try to detect **districts that are not already crowded with restaurants** and particularly interested in **areas with no Indian restaurants in vicinity**. Also analyze does this has any influence from **population density of foreigners**.

Power of data science is used to generate a few most promissing districts based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the district (any type of restaurant)
* number of Indian restaurants in the district, if any
* population density of foreigners in the district

Following data sources will be needed to extract/generate the required information:
* The districts in Munich and the population data is obtained from [this webpage](https://www.total-munich.com/20160623888/blog/moving-to-munich/moving-to-munich-introduction-to-munich-s-boroughs.html).
* I have exracted the `GeoJSON` data for the districts and is available at the following [link](https://s3.us.cloud-object-storage.appdomain.cloud/coursera-ibm-datascience/Munich_Districts.geojson)
* Centers of candidate districts will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **GeoPy Python package**
* Number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**


### Districts in Munich and population data

Let's read the data into Pandas dataframe and visualize the districts on the map. GeoPy is used to convert the district's center address to corresponding latitude & longitude. And also convert latitude & longitude to corresponding address.

GeoJSON data is used to draw the borders of the districts.

In [2]:
# If GeoPy package isn't installed, install using the below command
!pip install geopy

In [3]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

geolocator = Nominatim(user_agent="munich_explorer")

def get_coordinates(address):
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

address = 'Munich, Germany'
munich_center = get_coordinates(address)
print('Coordinate of {}: {}'.format(address, munich_center))


def get_address(latitude, longitude):
    try:
        location = geolocator.reverse((latitude, longitude))
        return location.address
    except:
        return None

addr = get_address(munich_center[0], munich_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(munich_center[0], munich_center[1], addr))

Coordinate of Munich, Germany: (48.1371079, 11.5753822)
Reverse geocoding check
-----------------------
Address of [48.1371079, 11.5753822] is: Marienplatz, Bezirksteil Angerviertel, Stadtbezirk 01 Altstadt-Lehel, München, Obb, Bayern, 80331, Deutschland


#### Loading Munich's districts and their corresponding data.

These are the location candidates around which the location to open an Indian restaurant is investigated.

In [4]:
# install 'lxml' python package to be able to read table HTML tables from Pandas
!pip install lxml

import requests

munich_districts_info = 'http://www.total-munich.com/20160623888/blog/moving-to-munich/moving-to-munich-introduction-to-munich-s-boroughs.html'
data = requests.get(munich_districts_info)

# loading the html table to pandas dataframe
districts = pd.read_html(data.text, header=0)[0]

# remove unwanted rows
districts = districts[:25]
districts.columns = [column.replace(' ', '') for column in districts.columns]
districts.rename(columns={'No.': 'District_Number'}, inplace=True)

# rectifying the mistakes in district names and type casting of few columns
districts.loc[1, 'District'] = districts.loc[1, 'District'].replace(' ', '')
districts.loc[18, 'District'] = districts.loc[18, 'District'].replace(' ', '')
districts.loc[:, 'District_Number'] = districts['District_Number'].astype(int)
districts.loc[:, 'Density(Persons/km²)'] = districts['Density(Persons/km²)'].astype(int)

# convrting percentage to actual numbers
foreigners_population = districts['Foreigners(%)'] * districts['Population'] / 100
districts.loc[:, 'Foreigners_Population'] = foreigners_population.astype(int)

foreigners_density = districts['Foreigners(%)'] * districts['Density(Persons/km²)'] / 100
districts.loc[:, 'Foreigners_Density(Persons/km²)'] = foreigners_density.astype(int)

districts

,District_Number,District,Surface(km²),Population,Density(Persons/km²),Foreigners(%),Foreigners_Population,Foreigners_Density(Persons/km²)
0,1,Altstadt-Lehel,3.15,20806,6614,25.8,5367,1706
1,2,Ludwigsvorstadt-Isarvorstadt,4.40,52177,11855,30.2,15757,3580
2,3,Maxvorstadt,4.30,52575,12233,26.0,13669,3180
3,4,Schwabing-West,4.36,67094,15378,22.3,14961,3429
4,5,Au-Haidhausen,4.22,60756,14399,23.4,14216,3369
5,6,Sendling,3.94,40422,10263,26.7,10792,2740
6,7,Sendling-Westpark,7.81,56485,7228,27.2,15363,1966
7,8,Schwanthalerhöhe,2.07,30083,14531,34.0,10228,4940
8,9,Neuhausen-Nymphenburg,12.91,97521,7551,23.9,23307,1804
9,10,Moosach,11.09,52657,4747,29.6,15586,1405


#### Adding latitude & longitude info for each districts.

In [5]:
districts = districts.join(pd.DataFrame(districts.loc[:, 'District'].apply(lambda x: get_coordinates(f'{x}, Munich, Germany')).tolist(), columns=['Latitude', 'Longitude']))
districts.head()

,District_Number,District,Surface(km²),Population,Density(Persons/km²),Foreigners(%),Foreigners_Population,Foreigners_Density(Persons/km²),Latitude,Longitude
0,1,Altstadt-Lehel,3.15,20806,6614,25.8,5367,1706,48.137828,11.574582
1,2,Ludwigsvorstadt-Isarvorstadt,4.40,52177,11855,30.2,15757,3580,48.131771,11.555809
2,3,Maxvorstadt,4.30,52575,12233,26.0,13669,3180,48.146570,11.571445
3,4,Schwabing-West,4.36,67094,15378,22.3,14961,3429,48.164417,11.570364
4,5,Au-Haidhausen,4.22,60756,14399,23.4,14216,3369,48.128753,11.590536


#### Let's download the GeoJSON file needed to plot districts in Munich.

In [6]:
# download countries geojson file
!wget --quiet https://s3.us.cloud-object-storage.appdomain.cloud/coursera-ibm-datascience/Munich_Districts.geojson -O Munich_Districts.geojson
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [7]:
# If folium package isn't installed, install using the below command
!pip install folium

     |████████████████████████████████| 92kB 19.9MB/s eta 0:00:01


### Plotting the disticts of Munich and their corresponding foreigners population density.

In [8]:
import folium

def get_munich_choropleth():
    latitude = munich_center[0]
    longitude = munich_center[1]

    munich_map = folium.Map(location=[latitude, longitude], zoom_start=12)
    munich_geo = r'Munich_Districts.geojson'

    choropleth = folium.Choropleth(
        geo_data=munich_geo,
        data=districts,
        columns=['District', 'Density(Persons/km²)'],
        key_on='feature.properties.DISTRICT',
        fill_color='YlOrRd', 
        fill_opacity=0.7, 
        line_opacity=0.2,
        legend_name='Population density in Munich districts',
        reset=True,
        highlight=True
    ).add_to(munich_map)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['DISTRICT'])
    )

    folium.Marker(munich_center, popup='Munich').add_to(munich_map)

    return munich_map

get_munich_choropleth()

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category. We'll make sure to detect and include all the subcategories of specific 'Indian restaurant' category, as we need info on Indian restaurants in the neighborhood. And also only the candidates around 1 km radius from the center of each district is considered.

Foursquare credentials are defined in hidden cell bellow.

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
# Category IDs corresponding to Indian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

indian_restaurant_categories = ['4bf58dd8d48988d10f941735', '54135bf5e4b08f3d2429dfe5', '54135bf5e4b08f3d2429dff3',
                                '54135bf5e4b08f3d2429dff5', '54135bf5e4b08f3d2429dfe2', '54135bf5e4b08f3d2429dff2',
                                '54135bf5e4b08f3d2429dfe1', '54135bf5e4b08f3d2429dfe3', '54135bf5e4b08f3d2429dfe8',
                                '54135bf5e4b08f3d2429dfe9', '54135bf5e4b08f3d2429dfe6', '54135bf5e4b08f3d2429dfdf',
                                '54135bf5e4b08f3d2429dfe4', '54135bf5e4b08f3d2429dfe7', '54135bf5e4b08f3d2429dfea',
                                '54135bf5e4b08f3d2429dfeb', '54135bf5e4b08f3d2429dfed', '54135bf5e4b08f3d2429dfee',
                                '54135bf5e4b08f3d2429dff4', '54135bf5e4b08f3d2429dfe0', '54135bf5e4b08f3d2429dfdd',
                                '54135bf5e4b08f3d2429dff6', '54135bf5e4b08f3d2429dfef', '54135bf5e4b08f3d2429dff0',
                                '54135bf5e4b08f3d2429dff1', '54135bf5e4b08f3d2429dfde', '54135bf5e4b08f3d2429dfec']

def is_indian_restaurant(categories, specific_filter=None):
    restaurant = True
    specific = False
    for c in categories:
        category_id = c[1]
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20190912'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
        print('exception')
    return venues

In [11]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found Indian restaurants

import pickle
from math import sqrt

def get_restaurants(lats, lons, area):
    restaurants = {}
    indian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon, area in zip(lats, lons, area):
        # Using radius=1km to get restaurants (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=1000, limit=1000)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_indian = is_indian_restaurant(venue_categories, specific_filter=indian_restaurant_categories)
            if is_res:
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_indian)
                area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_indian:
                    indian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, indian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
indian_restaurants = {}
location_restaurants = []
loaded = False

try:
    with open('restaurants.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('indian_restaurants.pkl', 'rb') as f:
        indian_restaurants = pickle.load(f)
    with open('location_restaurants.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, indian_restaurants, location_restaurants = get_restaurants(districts['Latitude'].tolist(), districts['Longitude'].tolist(), districts['Surface(km²)'].tolist())
    
    # Let's persists this in local file system
    with open('restaurants.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('indian_restaurants.pkl', 'wb') as f:
        pickle.dump(indian_restaurants, f)
    with open('location_restaurants.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [12]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Indian restaurants:', len(indian_restaurants))
print('Percentage of Indian restaurants: {:.2f}%'.format(len(indian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1076
Total number of Indian restaurants: 25
Percentage of Indian restaurants: 2.32%
Average number of restaurants in neighborhood: 46.0


In [13]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('5238825911d2715c80955a54', 'Augustiner Klosterwirt', 48.138649448058345, 11.572527009723416, 'Augustinerstr. 1, 80331 München', 885, False)
('4b58d627f964a520d66d28e3', 'Kilians', 48.13871994658476, 11.574555957384717, 'Frauenplatz 11, 80331 München', 903, False)
('4c65358a94160f47eab52e17', 'Chocolaterie Beluga', 48.135749958121224, 11.575775742530823, 'Viktualienmarkt 6, 80331 München', 247, False)
('56e15a45498eae4657e9617c', 'Bite Delite', 48.13999643231038, 11.57507175739173, 'Maffeistraße 6 (Fünf Höfe), 80333 München', 779, False)
('4ade0ca0f964a5202a6821e3', 'Andechser am Dom', 48.138302, 11.573778, 'Frauenplatz 7, 80331 München', 936, False)
('5aa40abaa22db77cd6026bf3', 'Leger am Dom', 48.138262321999065, 11.572932468978577, 'Liebfrauenstraße (Frauenplatz), 80331 München', 131, False)
('4b1fe8b3f964a520a02a24e3', 'ARAN Fünf Höfe', 48.1407145021652, 11.576498919834604, 'Theatinerstr. 12, 80333 München', 752, False)
('4b7802fcf964

In [14]:
print('List of Indian restaurants')
print('---------------------------')
for r in list(indian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(indian_restaurants))

List of Indian restaurants
---------------------------
('4b4db542f964a52010d626e3', 'Sangeet', 48.143081975981836, 11.575388983225329, 'Brienner Str. 10, 80333 München', 486, True)
('4cb5f3b49c7ba35dd2468e06', 'DHABA', 48.162666, 11.57483, 'Belgradstr. 16 (Herzogstr.), 80796 München', 384, True)
('4bbdc778a0a0c9b60bf41b0f', 'Bombay Tandoori', 48.12780336925669, 11.595254217270467, 'Rosenheimer Str. 75 (Metzstr.), 81667 München', 366, True)
('4b104ba6f964a520316d23e3', 'Shalimar', 48.13196044377003, 11.582532042619418, 'Morassistr. 16, 80469 München', 693, True)
('4bb33e3f4019a593e2a737b8', 'Goa Indian Restaurant', 48.13419460779889, 11.584111027422885, 'Thierschstr. 8, 80538 München', 771, True)
('4b695ed6f964a5203aa02be3', 'Safran', 48.129448671390776, 11.594737436253405, 'Steinstr. 79 (Rosenheimer Platz/Haidhausen), 81667 München', 321, True)
('4b068c90f964a52002ed22e3', 'Ganesha', 48.1265784260598, 11.597448700852, 'Rosenheimer Straße 113, 81669 München', 567, True)
('4bc3080dabf495

In [15]:
print('Restaurants around location')
print('---------------------------')
for i in range(20, 25):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 21: thai Streetkitchen, essence, Lemontrees II, Aran Brotgenuss & Kaffeekult, Rischart, Nihao, Viet Village, Schweizer Hof
Restaurants around location 22: Restaurant Apollon in "Die Lüfte", Bäckerei Reicherzer, Rackls Backstubn, Burenwirt, Aubinger Herzl, Bäcker Mayer, Shiv, Bäcker Mayer
Restaurants around location 23: Bäckerei Schuhmair, Zur Allacher Mühle, Würmtalhof, Gasthaus zur Schießstätte, Sicilia, Antica Osteria, Vietnam Today - Euro Asia Mini Markt, Bäckerei
Restaurants around location 24: Beerencafe Hofreiter, El Greco, Feldmochinger Hof, Rackls Backstubn, Taverna Wrossis, Schneid Kaffee, Wimmer Backerei, Rackl's Backstub'n
Restaurants around location 25: Potlatsch, Zöttl Bäckerei und Cafe, Gasthaus Gotthardgarten, Taverna (TAVERNA), Sabatono, Asien Perle, Laimers, Bistro No.2


Let's now see all the collected restaurants in our area of interest on map, and let's also show Indian restaurants in different color.

In [16]:
munich_map = get_munich_choropleth()

for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_indian = res[6]
    color = 'red' if is_indian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(munich_map)
munich_map

Looking good. So now we have all the restaurants in area within 1 km from center of each districts, and we know which ones are Indian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

It seems to appear that the more densly populated area have more food related businesses.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Indian restaurant!

## Methodology <a name="methodology"></a>

In this project the efforts are made on detecting areas in districts of Munich that have low restaurant density, particularly those with low number of Indian restaurants. We will limit our analysis to area 1km around center of each district.

In first step we have collected the required **data: location and type (category) of every restaurant within 1km from each distict's center**. We have also **identified Indian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas - we will use **heatmaps** to identify a few promising areas close to center of districts with low number of restaurants in general (*and* no Indian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Indian restaurants in radius of 250 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [17]:
location_restaurants_count = [len(res) for res in location_restaurants]

districts['Restaurants_in_area'] = location_restaurants_count

print('Average number of restaurants in 1km radius from center of every district:', np.array(location_restaurants_count).mean())

districts.head()

Average number of restaurants in 1km radius from center of every district: 46.0


,District_Number,District,Surface(km²),Population,Density(Persons/km²),Foreigners(%),Foreigners_Population,Foreigners_Density(Persons/km²),Latitude,Longitude,Restaurants_in_area
0,1,Altstadt-Lehel,3.15,20806,6614,25.8,5367,1706,48.137828,11.574582,100
1,2,Ludwigsvorstadt-Isarvorstadt,4.40,52177,11855,30.2,15757,3580,48.131771,11.555809,100
2,3,Maxvorstadt,4.30,52575,12233,26.0,13669,3180,48.146570,11.571445,100
3,4,Schwabing-West,4.36,67094,15378,22.3,14961,3429,48.164417,11.570364,51
4,5,Au-Haidhausen,4.22,60756,14399,23.4,14216,3369,48.128753,11.590536,100


In [18]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

indian_latlons = [[res[2], res[3]] for res in indian_restaurants.values()]

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Munich districts** on our map.

In [19]:
from folium import plugins
from folium.plugins import HeatMap

def style(feature):
    return {'color': 'blue', 'fill': False }

map_munich = folium.Map(location=munich_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_munich) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

Let's create another heatmap map showing **heatmap/density of Indian restaurants** only.

In [20]:
map_munich = folium.Map(location=munich_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_munich) #cartodbpositron cartodbdark_matter
HeatMap(indian_latlons).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

This map is not so 'hot' (Indian restaurants represent a subset of ~2.5% of all food businesses in Munich) but it also indicates higher density of existing Indian restaurants in **Altstadt-Lehel** and **Neuhausen-Nymphenburg** districts, with **no Indian restaurant in Maxvorstadt and Liam**.

Based on this we will now focus our analysis to district *Maxvorstadt*. *Maxvorstadt* is a district with high population density and high density restaurants in the center. Surprisingly with no Indian restaurants. We will move the center of our area of interest and reduce it's size to have a radius of **1km**.

Let's define new, more narrow region of interest.

In [21]:
roi_center = districts.loc[2,:]
roi_center

District_Number                              3
District                           Maxvorstadt
Surface(km²)                               4.3
Population                               52575
Density(Persons/km²)                     12233
Foreigners(%)                               26
Foreigners_Population                    13669
Foreigners_Density(Persons/km²)           3180
Latitude                               48.1466
Longitude                              11.5714
Restaurants_in_area                        100
Name: 2, dtype: object

In [22]:
map_munich = folium.Map(location=munich_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_munich) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
folium.Marker((roi_center['Latitude'], roi_center['Longitude']), popup=f'ROI Center: {roi_center["District"]}').add_to(map_munich)
folium.Circle((roi_center['Latitude'], roi_center['Longitude']), radius=1000, color='white', fill=True, fill_opacity=0.4).add_to(map_munich)
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

Not bad - this nicely covers all the pockets of low restaurant density in center of Macvorstadt.

In [23]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Munich center longitude={}, latitude={}'.format(munich_center[1], munich_center[0]))
x, y = lonlat_to_xy(munich_center[1], munich_center[0])
print('Munich center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Berlin center longitude={}, latitude={}'.format(lo, la))

     |████████████████████████████████| 1.5MB 9.2MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 24.6MB/s eta 0:00:01
Coordinate transformation check
-------------------------------
Munich center longitude=11.5753822, latitude=48.1371079
Munich center UTM X=245233.0015954985, Y=5337212.936840251
Berlin center longitude=11.575382200000002, latitude=48.13710789999999


In [24]:
roi_center_x, roi_center_y = lonlat_to_xy(roi_center['Longitude'], roi_center['Latitude'])
roi_center_x, roi_center_y

(244987.02477305106, 5338277.536343259)

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [25]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 1000
roi_x_min = roi_center_x - 1000

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 1001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

362 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Indian restaurant**.

In [26]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x, res_y = lonlat_to_xy(res[3], res[2])
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x, res_y = lonlat_to_xy(res[3], res[2])
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_indian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, indian_restaurants)
    roi_indian_distances.append(distance)
print('done.')

Generating data on location candidates... done.


In [27]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Indian restaurant':roi_indian_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Indian restaurant
0,48.138207,11.566624,244587.024773,5.337364e+06,10,848.461602
1,48.138247,11.567966,244687.024773,5.337364e+06,13,771.150490
2,48.138287,11.569308,244787.024773,5.337364e+06,13,699.613512
3,48.138327,11.570650,244887.024773,5.337364e+06,15,635.802685
4,48.138367,11.571991,244987.024773,5.337364e+06,16,582.263722
5,48.138407,11.573333,245087.024773,5.337364e+06,16,542.048918
6,48.138447,11.574675,245187.024773,5.337364e+06,22,518.269251
7,48.138487,11.576017,245287.024773,5.337364e+06,25,513.214385
8,48.138527,11.577358,245387.024773,5.337364e+06,23,527.422973
9,48.138924,11.564560,244437.024773,5.337451e+06,8,929.469447


Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Indian restaurants in radius of 250 meters**.

In [28]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ind_distance = np.array(df_roi_locations['Distance to Indian restaurant']>=250)
print('Locations with no Indian restaurants within 200m:', good_ind_distance.sum())

good_locations = np.logical_and(good_res_count, good_ind_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 95
Locations with no Indian restaurants within 200m: 340
Locations with both conditions met: 95


Let's see how this looks on a map.

In [29]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_munich = folium.Map(location=(roi_center['Latitude'], roi_center['Longitude']), zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_munich)
HeatMap(restaurant_latlons).add_to(map_munich)
folium.Circle((roi_center['Latitude'], roi_center['Longitude']), radius=1000, color='white', fill=True, fill_opacity=0.6).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
folium.Marker((roi_center['Latitude'], roi_center['Longitude']), popup=f'ROI Center: {roi_center["District"]}').add_to(map_munich)

for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_munich) 
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

Looking good. We now have a bunch of locations fairly close to center of Maxvorstadt district, and we know that each of those locations has no more than two restaurants in radius of 250m, and no Indian restaurant closer than 250m. Any of those locations is a potential candidate for a new Indian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [30]:
map_munich = folium.Map(location=(roi_center['Latitude'], roi_center['Longitude']), zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
folium.Marker((roi_center['Latitude'], roi_center['Longitude']), popup=f'ROI Center: {roi_center["District"]}').add_to(map_munich)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_munich)
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Indian restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [31]:
from sklearn.cluster import KMeans

number_of_clusters = 10

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_munich = folium.Map(location=(roi_center['Latitude'], roi_center['Longitude']), zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_munich)
HeatMap(restaurant_latlons).add_to(map_munich)
folium.Circle((roi_center['Latitude'], roi_center['Longitude']), radius=1000, color='white', fill=True, fill_opacity=0.4).add_to(map_munich)
folium.Marker(munich_center, popup='Munich').add_to(map_munich)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=250, color='green', fill=True, fill_opacity=0.25).add_to(map_munich) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_munich)
folium.GeoJson(r'Munich_Districts.geojson', style_function=style,
               tooltip=folium.features.GeoJsonTooltip(['DISTRICT']),
               name='geojson').add_to(map_munich)
map_munich

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [32]:
candidate_area_addresses = []
munich_center_x, munich_center_y = lonlat_to_xy(munich_center[1], munich_center[0])
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(lat, lon).replace(', Germany', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, munich_center_x, munich_center_y)
    print('{}{} => {:.1f}km from Munich Center (Marienplatz)'.format(addr, ' '*(50-len(addr)), d/1000))

Addresses of centers of areas recommended for further analysis

Tillmann Kinder-und-Jugendhaus, 23, Kaulbachstraße, Schönfeldvorstadt, Bezirksteil Schönfeldvorstadt, Stadtbezirk 03 Maxvorstadt, München, Obb, Bayern, 80539, Deutschland => 1.3km from Munich Center (Marienplatz)
Arcisstraße, Bezirksteil Königsplatz, Stadtbezirk 03 Maxvorstadt, München, Obb, Bayern, 80333, Deutschland => 1.1km from Munich Center (Marienplatz)
2, Von-der-Tann-Straße, Schönfeldvorstadt, Bezirksteil Schönfeldvorstadt, Stadtbezirk 03 Maxvorstadt, München, Obb, Bayern, 80539, Deutschland => 1.0km from Munich Center (Marienplatz)
46c, Schleißheimer Straße, Bezirksteil Augustenstraße, Stadtbezirk 03 Maxvorstadt, München, Obb, Bayern, 80333, Deutschland => 1.9km from Munich Center (Marienplatz)
6, Max-Joseph-Straße, Bezirksteil Königsplatz, Stadtbezirk 03 Maxvorstadt, München, Obb, Bayern, 80333, Deutschland => 0.8km from Munich Center (Marienplatz)
Hofgarten, Hofgartenstraße, Bezirksteil Graggenau, Stadtbezirk 01

This concludes our analysis. We have created 10 addresses representing centers of zones containing locations with low number of restaurants and no Indian restaurants nearby, all zones being fairly close to district's center. Although zones are shown on map with a radius of ~250 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area in search for potential restaurant locations. Most of the zones are located in Maxvorstadt district, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

In [33]:
map_munich = folium.Map(location=(roi_center['Latitude'], roi_center['Longitude']), zoom_start=14)
folium.Circle(munich_center, popup='Munich Center (Marienplatz)', radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_munich)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_munich) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_munich)
map_munich

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Munich, there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected in Munich center, so we focused our attention to Maxvorstadt district. Another ditrict could be potentially interesting is Liam, but our attention was focused on Maxvorstadt which offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics *and* a number of pockets of low restaurant density.

After directing our attention to this more narrow area of interest we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 250m and those with an Indian restaurant closer than 250m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 10 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Munich center but not crowded with existing restaurants (particularly Indian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Munich areas close to center with low number of restaurants (particularly Indian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Indian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (Maxvorstadt), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of districts and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.